In [13]:
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame
import os
import pandas as pd
import hvplot.pandas
import datetime
#!pip install schedule
import schedule
from finta import TA
import numpy as np
import time
#!pip install ccxt
import ccxt
import warnings
warnings.filterwarnings('ignore')

#!pip install alpaca-py
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data import CryptoDataStream
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import matplotlib.pyplot as plt

from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
pd.set_option("display.max_rows", None)

from datetime import datetime
#and the access its now method simpler
d1 = datetime.now()
import hvplot

In [14]:
load_dotenv("key.env")
alpaca_api_key = os.getenv("Alpaca_Paper_Key_ID")
alpaca_secret_key = os.getenv("Alpaca_Paper_Secret_Key")


api = REST(
    key_id=alpaca_api_key,
    secret_key=alpaca_secret_key,
    base_url="https://paper-api.alpaca.markets")

In [15]:
# Define variables
symbol = 'BTCUSD'
qty_per_trade = 1
start_date = "2018-01-01"
end_date = datetime

In [16]:
from alpaca.trading.client import TradingClient

trading_client = TradingClient(alpaca_api_key, alpaca_secret_key)

account = trading_client.get_account()
account

{   'account_blocked': False,
    'account_number': 'PA3TFNVOZJQU',
    'accrued_fees': '0',
    'buying_power': '372890.24',
    'cash': '94627.4853175',
    'created_at': datetime.datetime(2022, 1, 5, 23, 28, 34, 385878, tzinfo=datetime.timezone.utc),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 1,
    'daytrading_buying_power': '372890.24',
    'equity': '94627.4853175',
    'id': UUID('a9b4a15f-d928-4cde-a5a1-585c66269d91'),
    'initial_margin': '0',
    'last_equity': '94684.12',
    'last_maintenance_margin': '1266.36',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '4',
    'non_marginable_buying_power': '91323.04',
    'pattern_day_trader': True,
    'pending_transfer_in': '0',
    'pending_transfer_out': None,
    'portfolio_value': '94627.4853175',
    'regt_buying_power': '189254.970635',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '94631.54',
    'status': <Ac

In [17]:
#Create Function to Check Account Positions
# Check Whether Account Currently Holds Symbol
def check_positions(symbol):
    in_positions = trading_client.get_all_positions()
    if symbol in str(in_positions):
        return True
    return False


in_position = check_positions(symbol)




In [18]:
#basic upper bands = (HIGH + LOW) / 2 + Multiolier * ATR
# basic lowerband = (HIGH + LOW) / 2 - Multiplier * ATR
def tr(df):
    df['previous_close'] = df.close.shift(1)
    df['high-low'] = df.high - df.close
    df['high-pc'] = abs(df.high - df.previous_close)
    df['low-pc'] = abs(df.low - df.previous_close)
    tr = df[['high-low', 'high-pc', 'low-pc']].max(axis=1)
    
    return tr





In [19]:
def atr(df, period=14):
    df['tr'] = tr(df)
    print("calculate average true range")
    the_atr = df.tr.rolling(period).mean()
    
    return the_atr



In [20]:
def supertrend(df, period=7, multiplier=3):
    print("calculating supertrend")
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] =  ((df.high + df.low) / 2) + (multiplier * df.atr)
    df['lowerband'] = ((df.high + df.low) / 2) - (multiplier * df.atr)
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

In [21]:
# Submit order
def buy():
    api.submit_order(
    symbol="BTCUSD", 
    qty=.001, 
    side='buy', 
    time_in_force="gtc" )

def sell():
    api.submit_order(
    symbol="BTCUSD", 
    qty=.001, 
    side='sell', 
    time_in_force="gtc" )

#limit_order_data = LimitOrderRequest(
#                    symbol="BTC/USD",
#                    limit_price=17000,
#                    notional=4000,
#                    side=OrderSide.SELL,
#                    time_in_force=TimeInForce.FOK
 #                  )

# Limit order
#limit_order = trading_client.submit_order(
#                order_data=limit_order_data
#             )

In [ ]:
def check_buy_sell_signals(df):
    global in_position
    
    print("checking for buy and sell")
    print(df.tail(5))
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1
    
    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("changed to uptrend, BUY!!")
        if not in_position:
            order = buy()
            print(order)
            in_position = True
        else:
            print("position open, nothing to do")
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position:
            print("changed  to downtrends, SELL!!")
            order = sell()
            print(order)
            in_position = False
        else:
            print("non position open, nothing to do")
    
    

def fetching_bars():
    print(f"Fetching new bars for {d1.isoformat()}")
    client = CryptoHistoricalDataClient()
    request_params = CryptoBarsRequest(
                        symbol_or_symbols=["BTC/USD"],
                        timeframe=TimeFrame.Minute,
                        limit=100)
    bars = client.get_crypto_bars(request_params)
    bars.df
    df = bars.df
    
    supertrend_data = supertrend(df)
    supertrend_data = pd.DataFrame(supertrend_data)
    check_buy_sell_signals(supertrend_data)
    
    
schedule.every(30).seconds.do(fetching_bars)


while True:
    schedule.run_pending()
    time.sleep(1)

Fetching new bars for 2022-09-29T10:41:39.918668
calculating supertrend
calculate average true range
checking for buy and sell
                                      open     high      low    close  volume  \
symbol  timestamp                                                               
BTC/USD 2022-09-29 06:42:00+00:00  19400.0  19400.0  19387.0  19387.0  3.8372   
        2022-09-29 06:43:00+00:00  19390.0  19390.0  19383.0  19389.0  6.6426   
        2022-09-29 06:44:00+00:00  19390.0  19409.0  19390.0  19409.0  5.0018   
        2022-09-29 06:45:00+00:00  19405.0  19405.0  19378.0  19378.0  0.1036   
        2022-09-29 06:46:00+00:00  19380.0  19390.0  19380.0  19390.0  2.5403   

                                   trade_count          vwap  previous_close  \
symbol  timestamp                                                              
BTC/USD 2022-09-29 06:42:00+00:00         12.0  19390.554519         19399.0   
        2022-09-29 06:43:00+00:00         17.0  19386.878075     